# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

geoapify_key

'095e971df39d49d2af1f09a49601ab74'

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,19.44,66,98,10.72,PN,1690595686
1,1,ribeira grande,38.5167,-28.7000,22.65,91,11,0.89,PT,1690595687
2,2,dapa,9.7594,126.0531,26.67,78,100,3.76,PH,1690595687
3,3,ilulissat,69.2167,-51.1000,0.26,92,39,2.42,GL,1690595687
4,4,malango,-2.9627,119.9001,25.44,59,33,1.76,ID,1690595687


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
# %%capture --no-display

# Configure the map plot
# YOUR CODE HERE

for index, row in city_data_df.iterrows():
    city_name = row['City']
    city_lat = row['Lat']
    city_lng = row['Lng']
    city_humidity = row['Humidity']

map_plot = city_data_df.hvplot.points(x='Lng', y='Lat', c='Humidity', tiles='OSM',cmap='viridis',size='Max Temp',geo=True,hover_cols=['City'])

map_plot
# Display the map
# YOUR CODE HERE

c:\Users\xxhh5\anaconda3\envs\PythonData\lib\site-packages\cartopy\crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
c:\Users\xxhh5\anaconda3\envs\PythonData\lib\site-packages\cartopy\crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
c:\Users\xxhh5\anaconda3\envs\PythonData\lib\site-packages\cartopy\crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
c:\Users\xxhh5\anaconda3\envs\PythonData\lib\site

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Max Temp,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [41]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
New_city_data=city_data_df[city_data_df['Cloudiness']==0]

null_values=New_city_data.isnull().count()
null_values
# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
New_city_data
# YOUR CODE HERE

City_ID       85
City          85
Lat           85
Lng           85
Max Temp      85
Humidity      85
Cloudiness    85
Wind Speed    85
Country       85
Date          85
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_data = {
    'City': New_city_data['City'],
    'Lat': New_city_data['Lat'],
    'Lng': New_city_data['Lng'],
    'Humidity': New_city_data['Humidity'],
    'Country': New_city_data['Country']
}

hotel_df = pd.DataFrame(hotel_data)

    
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name']=''
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
7,yanbu,24.0891,38.0637,89,SA,
12,ushuaia,-54.8000,-68.3000,57,AR,
22,brookings,44.3114,-96.7984,61,US,
32,bethel,41.3712,-73.4140,84,US,
38,port lincoln,-34.7333,135.8667,59,AU,
...,...,...,...,...,...,...
564,tremp,42.1670,0.8949,69,ES,
578,bani walid,31.7566,13.9942,65,LY,
585,atakent,40.8464,68.5067,27,KZ,
586,moyynkum,44.2914,72.9469,28,KZ,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [52]:
# Set parameters to search for a hotel



# Iterate through the hotel_df DataFrame
radius = 10000
categories = 'accommodation.hotel'
limit = 20
# Print a message to start the hotel search
# Set base URL for the API request
base_url = "https://api.geoapify.com/v2/places"
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    Lat = row['Lat']
    Lng = row['Lng']
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f"circle:{Lat},{Lng},{radius}"
    bias = f"proximity:{Lat},{Lng}"
    params = {
        "categories": categories,
        "filters": filters,
        "bias": bias,
        "limit": limit,
        "apiKey": geoapify_key
    }
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    data = response.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_name = data["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except Exception as e:
        print(e)
    # Log the search results
    print(f"{row['City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display the DataFrame w
hotel_df

'features'
yanbu - nearest hotel: No hotel found
'features'
ushuaia - nearest hotel: No hotel found
'features'
brookings - nearest hotel: No hotel found
'features'
bethel - nearest hotel: No hotel found
'features'
port lincoln - nearest hotel: No hotel found
'features'
port elizabeth - nearest hotel: No hotel found
'features'
pecos - nearest hotel: No hotel found
'features'
anadyr - nearest hotel: No hotel found
'features'
tiksi - nearest hotel: No hotel found
'features'
luena - nearest hotel: No hotel found
'features'
eagle - nearest hotel: No hotel found
'features'
baymak - nearest hotel: No hotel found
'features'
corinto - nearest hotel: No hotel found
'features'
carnarvon - nearest hotel: No hotel found
'features'
crane - nearest hotel: No hotel found
'features'
ceres - nearest hotel: No hotel found
'features'
toliara - nearest hotel: No hotel found
'features'
fortuna - nearest hotel: No hotel found
'features'
superior - nearest hotel: No hotel found
'features'
protaras - nearest h

,City,Lat,Lng,Humidity,Country,Hotel Name
7,yanbu,24.0891,38.0637,89,SA,No hotel found
12,ushuaia,-54.8000,-68.3000,57,AR,No hotel found
22,brookings,44.3114,-96.7984,61,US,No hotel found
32,bethel,41.3712,-73.4140,84,US,No hotel found
38,port lincoln,-34.7333,135.8667,59,AU,No hotel found
...,...,...,...,...,...,...
564,tremp,42.1670,0.8949,69,ES,No hotel found
578,bani walid,31.7566,13.9942,65,LY,No hotel found
585,atakent,40.8464,68.5067,27,KZ,No hotel found
586,moyynkum,44.2914,72.9469,28,KZ,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)